In [ ]:
# Compute class weights: more weight to the minority class (1)
neg, pos = np.bincount(y_train_np.astype(int))
total = neg + pos
weight_for_0 = (1.0 / neg) * (total / 2.0)
weight_for_1 = (1.0 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
class_weight